### Try to find a way to compute the synchrony of neurons accross time

In [217]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy
from scipy.signal import savgol_filter

import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt
%matplotlib qt

import seaborn as sns 
import basefunction.bk_load as bk
import basefunction.anass_load as at

In [15]:
def binSpikes(neurons,binSize = 0.05,start = 0,stop = 0):
    if stop == 0:
        stop = np.max([neuron.as_units('s').index[-1] for neuron in neurons])

    bins = np.arange(start,stop,binSize)
    binned = []
    for neuron in neurons:
        hist,b = np.histogram(neuron.as_units('s').index,bins = bins)
        binned.append(hist)
    return np.array(binned)

def plotIntervals(intervals,col):
    for interval in intervals.as_units('s').values:
        plt.axvspan(interval[0],interval[1], facecolor=col, alpha=0.5)
        
def synchrony(neurons,binsize):
    print(len(neurons))
    print(binsize)
    return True

def main(path):
    neurons,metadata = bk.loadSpikeData(path)
    return neurons,metadata

In [233]:
path = bk.get_session()['Path'][5]
bk.set_current_session(path)

neurons,metadata = main(path)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Loading Rat08-20130713.clu.1
Loading Rat08-20130713.clu.2
Loading Rat08-20130713.clu.3
Loading Rat08-20130713.clu.4
Loading Rat08-20130713.clu.5
Loading Rat08-20130713.clu.6
Loading Rat08-20130713.clu.7
Loading Rat08-20130713.clu.8
Loading Rat08-20130713.clu.9
Loading Rat08-20130713.clu.10
Loading Rat08-20130713.clu.11
Loading Rat08-20130713.clu.12
Loading Rat08-20130713.clu.13
Loading Rat08-20130713.clu.14
Loading Rat08-20130713.clu.15
Loading Rat08-20130713.clu.16
Loading Rat08-20130713.clu.17
Loading Rat08-20130713.clu.18
Loading Rat08-20130713.clu.19
Loading Rat08-20130713.clu.20


In [270]:
binSize = 0.1
winSize = int(1/binSize)
step = int(1/binSize)

binned = binSpikes(neurons,binSize,start = 0,stop = 25000)

pbar = ProgressBar()
corr = np.empty((int(binned.shape[1]/step)+1,len(neurons),len(neurons)))   

for i,j in enumerate(pbar(range(0,binned.shape[1],step))):
    corr[i] = np.corrcoef(binned[:,j:j+winSize])
    corr[i][np.isnan(corr[i])] = 0

# corr = np.abs(corr)

stru = ['Hpc','BLA','CeCM']
stru_shank = {}
stru_mask  = {}
stru_corr  = {}
stru_sync  = {}

for s in stru:
    stru_shank.update({s:at.get_structure_shanks(bk.session,s)})
    if not np.any(stru_shank[s]):
        print('Could not compute ' + s + ' because is you don\'t have shank in it')
        continue
    stru_mask.update({s:[i in stru_shank[s] for i in metadata[:,0]]})
    
    
for s in stru_mask.keys():
    print('computing '+s)
    stru_corr.update({s:corr[:,:,stru_mask[s]][:,stru_mask[s]]})
    stru_sync.update({s:np.nanmean(stru_corr[s],axis = (1,2))})

stru_corr.update({'Hpc-BLA':corr[:,:,stru_mask['Hpc']][:,stru_mask['BLA']]})
stru_sync.update({'Hpc-BLA':np.nanmean(stru_corr['Hpc-BLA'],axis = (1,2))})
# for s,s2 in itertools.combinations(stru_mask.keys(),2):
#     print('computing '+s+'-'+s2)
#     stru_corr.update({s+'-'+s2:corr[:,:,stru_mask[s]][:,stru_mask[s2]]})
#     stru_sync.update({s+'-'+s2:np.nanmean(stru_corr[s+'-'+s2],axis = (1,2))})

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100% |########################################################################|


Could not compute CeCM because is you don't have shank in it
computing Hpc
computing BLA


In [242]:
plt.figure()
# plt.plot(savgol_filter(stru_sync['BLA'],201,3))
# plt.plot(savgol_filter(bla_mean,201,3))
# plt.plot(savgol_filter(cecm_mean,201,3))
# plt.plot(savgol_filter(hpc_bla_mean,201,3))

for s in stru_sync:
    print(s)
    plt.plot(savgol_filter(stru_sync[s],201,3))

plt.title('synchrony during ' + bk.session)
plt.legend([s for s in stru_sync.keys()])
plt.ylabel('r')
plt.xlabel('Time(s)')

Rem = nts.IntervalSet(scipy.io.loadmat('States.mat')['Rem'][:,0],scipy.io.loadmat('States.mat')['Rem'][:,1],time_units = 's')
plotIntervals(Rem,'darkorange')

sws = nts.IntervalSet(scipy.io.loadmat('States.mat')['sws'][:,0],scipy.io.loadmat('States.mat')['sws'][:,1],time_units = 's')
plotIntervals(sws,'grey')


Hpc
BLA
Hpc-BLA


In [294]:
plt.figure()
plt.subplot(1,3,1)
plt.imshow(stru_corr['Hpc'][21000,:,:])
plt.clim(-1,1)
plt.subplot(1,3,2)
plt.imshow(np.abs(stru_corr['Hpc'][21000,:,:]))
plt.clim(-1,1)
plt.subplot(1,3,3)
plt.colorbar()



In [363]:
plt.figure()
plt.eventplot(a[stru_mask['Hpc']],color = 'black')
plt.ylabel('Neurons')
plt.xlabel('Time(s)')


Text(0.5, 0, 'Time(s)')

In [361]:
neurons[1].as_units('s').index[0:3]
a = []
for i in neurons:
    a.append(i.as_units('s').loc[5:8].index)
    
a = np.array(a,dtype = 'object')

In [334]:
# for i in corr:
#     plt.clf()
#     sns.heatmap(i,square = True,vmin = 0,vmax = 0.5,cmap = 'viridis')
#     plt.pause(0.001)


In [7]:
for i in range(0,binned.shape[1],step):
    t = time.time()
    plt.clf()
    corr = np.corrcoef(binned[:,i:i+winSize])
    corr[np.isnan(corr)] = 0
    sns.heatmap(corr,square = True,vmin = 0,vmax = 0.1,cmap = 'viridis')
    plt.draw()
    plt.title(i)
    plt.pause(0.001)
    print(time.time()-t)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


1.3388242721557617
0.6753783226013184
0.6000332832336426
0.633859395980835
0.6082172393798828
0.6475844383239746
0.6454768180847168
0.6423957347869873
0.6567230224609375
0.6349830627441406
0.6251139640808105
0.6436095237731934
0.6054158210754395
0.656491756439209
0.6271243095397949
0.622647762298584
0.6530306339263916
0.6557021141052246
0.652904748916626
0.6745750904083252
0.6042311191558838
0.6158194541931152
0.66536545753479
0.5854179859161377
0.6224005222320557
0.6274163722991943
0.6218991279602051
0.6695849895477295
0.6212165355682373
0.635246753692627
0.6423590183258057
0.5898680686950684
0.6723871231079102
0.6828391551971436
0.6376233100891113
0.6347110271453857
0.5967564582824707
0.6393465995788574
0.6409778594970703
0.6371846199035645
0.6230118274688721
0.6440868377685547
0.6245479583740234
0.6640944480895996
0.6246001720428467
0.5836658477783203
0.6571216583251953
0.649038553237915
0.5839204788208008
0.6620240211486816
0.6092174053192139
0.6659440994262695
0.6448111534118652
0

KeyboardInterrupt: 